# Clase de Data Science: Creacion de APIs con Flask

## Objetivo del Curso
Aprender a crear APIs RESTful con Flask desde cero, incluyendo endpoints basicos y manejo de bases de datos.

## Documentacion Oficial
- Flask: https://flask.palletsprojects.com/
- Flask Quickstart: https://flask.palletsprojects.com/en/3.0.x/quickstart/
- Flask API: https://flask.palletsprojects.com/en/3.0.x/api/
- SQLite con Python: https://docs.python.org/3/library/sqlite3.html

## Estructura del Curso
1. Introduccion a Flask y configuracion inicial
2. Endpoints basicos (GET, POST, PUT, DELETE)
3. Manejo de parametros y JSON
4. Integracion con base de datos SQLite
5. Operaciones CRUD completas

---

## 1. Instalacion de Dependencias

Primero necesitamos instalar Flask. Ejecuta el siguiente comando:

In [ ]:
# Instalacion de Flask
# Flask es un micro-framework web para Python
# Documentacion: https://flask.palletsprojects.com/en/3.0.x/installation/
!pip install flask

## 2. Primera Aplicacion Flask - Hola Mundo

Vamos a crear nuestra primera aplicacion Flask muy simple para entender los conceptos basicos.

In [ ]:
# Importamos Flask desde el paquete flask
from flask import Flask

# Creamos una instancia de la aplicacion Flask
# __name__ ayuda a Flask a saber donde buscar recursos (templates, archivos estaticos, etc.)
app = Flask(__name__)

# Definimos una ruta usando el decorador @app.route()
# Esto significa que cuando alguien visite '/' (la raiz del sitio), se ejecutara esta funcion
@app.route('/')
def hola_mundo():
    # La funcion retorna el contenido que se mostrara en el navegador
    return 'Hola Mundo! Bienvenido a tu primera API con Flask'

print("Aplicacion Flask creada exitosamente!")
print("Para ejecutarla, copia este codigo a app.py y ejecuta: python app.py")

### Explicacion del codigo:
- `Flask(__name__)`: Crea la aplicacion Flask
- `@app.route('/')`: Decorador que define la URL que activara la funcion
- La funcion retorna lo que el usuario vera en su navegador

**COPIA EL SIGUIENTE CODIGO A `app.py` PARA EJECUTAR LA APLICACION:**

In [ ]:
# CODIGO PARA app.py - VERSION 1: Hola Mundo
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hola_mundo():
    return 'Hola Mundo! Bienvenido a tu primera API con Flask'

# Este bloque solo se ejecuta si corremos este archivo directamente
# (no si lo importamos desde otro lugar)
if __name__ == '__main__':
    # debug=True permite ver errores detallados y reiniciar automaticamente al guardar cambios
    # En produccion debe ser False por seguridad
    app.run(debug=True, host='0.0.0.0', port=5000)

---

## 3. Endpoints Basicos - Metodos HTTP

Las APIs REST utilizan diferentes metodos HTTP para diferentes operaciones:
- **GET**: Obtener datos (lectura)
- **POST**: Crear nuevos datos
- **PUT**: Actualizar datos existentes
- **DELETE**: Eliminar datos

Documentacion: https://flask.palletsprojects.com/en/3.0.x/quickstart/#http-methods

### 3.1 Endpoint GET - Obtener Informacion

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

# Simulamos una base de datos en memoria con una lista de diccionarios
# En una aplicacion real, esto vendria de una base de datos
productos = [
    {'id': 1, 'nombre': 'Laptop', 'precio': 1200},
    {'id': 2, 'nombre': 'Mouse', 'precio': 25},
    {'id': 3, 'nombre': 'Teclado', 'precio': 75}
]

# Endpoint para obtener todos los productos
# Por defecto, @app.route() solo acepta peticiones GET
@app.route('/api/productos', methods=['GET'])
def obtener_productos():
    # jsonify() convierte datos de Python (listas, diccionarios) a formato JSON
    # JSON es el formato estandar para intercambiar datos en APIs
    return jsonify(productos)

# Endpoint para obtener un producto especifico por ID
# <int:id> captura un valor numerico de la URL y lo pasa como parametro
@app.route('/api/productos/<int:id>', methods=['GET'])
def obtener_producto(id):
    # Buscamos el producto con el id especificado
    # next() devuelve el primer elemento que cumple la condicion, o None si no hay ninguno
    producto = next((p for p in productos if p['id'] == id), None)
    
    if producto:
        return jsonify(producto)
    else:
        # Si no encontramos el producto, devolvemos un error 404 (Not Found)
        # El segundo valor es el codigo de estado HTTP
        return jsonify({'error': 'Producto no encontrado'}), 404

print("Endpoints GET configurados!")

### 3.2 Endpoint POST - Crear Nuevos Recursos

In [ ]:
from flask import Flask, jsonify, request

# request es un objeto global que contiene todos los datos de la peticion HTTP
# Documentacion: https://flask.palletsprojects.com/en/3.0.x/api/#flask.request

# Endpoint para crear un nuevo producto
@app.route('/api/productos', methods=['POST'])
def crear_producto():
    # request.get_json() obtiene los datos JSON enviados en el cuerpo de la peticion
    # force=True permite procesar JSON incluso si el Content-Type no es application/json
    nuevo_producto = request.get_json()
    
    # Validamos que se hayan enviado los campos requeridos
    if not nuevo_producto or 'nombre' not in nuevo_producto or 'precio' not in nuevo_producto:
        # 400 = Bad Request (peticion mal formada)
        return jsonify({'error': 'Faltan datos requeridos: nombre y precio'}), 400
    
    # Generamos un nuevo ID (en este caso, el siguiente numero disponible)
    # En una base de datos real, esto se maneja automaticamente
    nuevo_id = max(p['id'] for p in productos) + 1 if productos else 1
    
    # Creamos el nuevo producto con los datos recibidos
    producto = {
        'id': nuevo_id,
        'nombre': nuevo_producto['nombre'],
        'precio': nuevo_producto['precio']
    }
    
    # Añadimos el producto a nuestra lista
    productos.append(producto)
    
    # 201 = Created (recurso creado exitosamente)
    return jsonify(producto), 201

print("Endpoint POST configurado!")

### 3.3 Endpoint PUT - Actualizar Recursos Existentes

In [ ]:
# Endpoint para actualizar un producto existente
@app.route('/api/productos/<int:id>', methods=['PUT'])
def actualizar_producto(id):
    # Buscamos el producto a actualizar
    producto = next((p for p in productos if p['id'] == id), None)
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    # Obtenemos los nuevos datos del cuerpo de la peticion
    datos_actualizados = request.get_json()
    
    # Actualizamos solo los campos que se enviaron
    # get() devuelve el valor si existe, o el valor actual si no se envio
    producto['nombre'] = datos_actualizados.get('nombre', producto['nombre'])
    producto['precio'] = datos_actualizados.get('precio', producto['precio'])
    
    # 200 = OK (operacion exitosa)
    return jsonify(producto), 200

print("Endpoint PUT configurado!")

### 3.4 Endpoint DELETE - Eliminar Recursos

In [ ]:
# Endpoint para eliminar un producto
@app.route('/api/productos/<int:id>', methods=['DELETE'])
def eliminar_producto(id):
    # Usamos global para modificar la lista de productos del scope exterior
    global productos
    
    # Verificamos que el producto existe
    producto = next((p for p in productos if p['id'] == id), None)
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    # Filtramos la lista para eliminar el producto con el id especificado
    productos = [p for p in productos if p['id'] != id]
    
    # 204 = No Content (operacion exitosa sin contenido que devolver)
    # Es comun en DELETE no devolver ningun contenido
    return '', 204

print("Endpoint DELETE configurado!")

**COPIA EL SIGUIENTE CODIGO A `app.py` PARA TENER TODOS LOS ENDPOINTS BASICOS:**

In [ ]:
# CODIGO PARA app.py - VERSION 2: CRUD Basico en Memoria
from flask import Flask, jsonify, request

app = Flask(__name__)

# Base de datos simulada en memoria
productos = [
    {'id': 1, 'nombre': 'Laptop', 'precio': 1200},
    {'id': 2, 'nombre': 'Mouse', 'precio': 25},
    {'id': 3, 'nombre': 'Teclado', 'precio': 75}
]

@app.route('/')
def inicio():
    return 'API de Productos - Endpoints disponibles: /api/productos'

# GET: Obtener todos los productos
@app.route('/api/productos', methods=['GET'])
def obtener_productos():
    return jsonify(productos)

# GET: Obtener un producto por ID
@app.route('/api/productos/<int:id>', methods=['GET'])
def obtener_producto(id):
    producto = next((p for p in productos if p['id'] == id), None)
    if producto:
        return jsonify(producto)
    return jsonify({'error': 'Producto no encontrado'}), 404

# POST: Crear un nuevo producto
@app.route('/api/productos', methods=['POST'])
def crear_producto():
    nuevo_producto = request.get_json()
    
    if not nuevo_producto or 'nombre' not in nuevo_producto or 'precio' not in nuevo_producto:
        return jsonify({'error': 'Faltan datos requeridos: nombre y precio'}), 400
    
    nuevo_id = max(p['id'] for p in productos) + 1 if productos else 1
    
    producto = {
        'id': nuevo_id,
        'nombre': nuevo_producto['nombre'],
        'precio': nuevo_producto['precio']
    }
    
    productos.append(producto)
    return jsonify(producto), 201

# PUT: Actualizar un producto existente
@app.route('/api/productos/<int:id>', methods=['PUT'])
def actualizar_producto(id):
    producto = next((p for p in productos if p['id'] == id), None)
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    datos_actualizados = request.get_json()
    producto['nombre'] = datos_actualizados.get('nombre', producto['nombre'])
    producto['precio'] = datos_actualizados.get('precio', producto['precio'])
    
    return jsonify(producto), 200

# DELETE: Eliminar un producto
@app.route('/api/productos/<int:id>', methods=['DELETE'])
def eliminar_producto(id):
    global productos
    
    producto = next((p for p in productos if p['id'] == id), None)
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    productos = [p for p in productos if p['id'] != id]
    return '', 204

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

### Probando los Endpoints

Puedes probar los endpoints de estas formas:
1. Usando **curl** desde la terminal
2. Usando **Postman** o **Insomnia** (herramientas graficas)
3. Usando **requests** de Python

Ejemplos con curl:

In [ ]:
# Ejemplos de comandos para probar la API (ejecutar en terminal, no aqui)

# GET: Obtener todos los productos
# curl http://localhost:5000/api/productos

# GET: Obtener un producto especifico
# curl http://localhost:5000/api/productos/1

# POST: Crear un nuevo producto
# curl -X POST http://localhost:5000/api/productos -H "Content-Type: application/json" -d "{\"nombre\":\"Monitor\",\"precio\":300}"

# PUT: Actualizar un producto
# curl -X PUT http://localhost:5000/api/productos/1 -H "Content-Type: application/json" -d "{\"precio\":1100}"

# DELETE: Eliminar un producto
# curl -X DELETE http://localhost:5000/api/productos/2

print("Comandos de ejemplo listados arriba")

---

## 4. Integracion con Base de Datos SQLite

Hasta ahora hemos trabajado con datos en memoria que se pierden al reiniciar la aplicacion.
Ahora vamos a integrar una base de datos SQLite para persistir los datos.

**SQLite** es una base de datos ligera que no requiere servidor, ideal para desarrollo y proyectos pequeños.

Documentacion:
- SQLite en Python: https://docs.python.org/3/library/sqlite3.html
- Flask y SQLite: https://flask.palletsprojects.com/en/3.0.x/patterns/sqlite3/

### 4.1 Configuracion Inicial de la Base de Datos

In [ ]:
import sqlite3
from flask import Flask, g

# Nombre del archivo de base de datos
DATABASE = 'productos.db'

# Funcion para obtener la conexion a la base de datos
# g es un objeto especial de Flask que almacena datos durante una peticion
# Permite reutilizar la misma conexion en multiples funciones durante una peticion
def get_db():
    # Si no existe una conexion en g, la creamos
    if 'db' not in g:
        # Conectamos a la base de datos SQLite
        g.db = sqlite3.connect(DATABASE)
        # Row permite acceder a las columnas por nombre ademas de por indice
        g.db.row_factory = sqlite3.Row
    return g.db

# Funcion para cerrar la conexion al final de cada peticion
# @app.teardown_appcontext se ejecuta al finalizar el contexto de la aplicacion
def close_db(error):
    # Obtenemos la conexion de g, o None si no existe
    db = g.pop('db', None)
    if db is not None:
        db.close()

print("Funciones de conexion a base de datos creadas!")

### 4.2 Inicializacion de la Base de Datos - Crear Tablas

In [ ]:
# Funcion para inicializar la base de datos
# Crea las tablas necesarias si no existen
def init_db():
    # Conectamos directamente (fuera del contexto de Flask)
    db = sqlite3.connect(DATABASE)
    cursor = db.cursor()
    
    # SQL para crear la tabla de productos
    # INTEGER PRIMARY KEY AUTOINCREMENT: genera IDs automaticamente
    # NOT NULL: el campo es obligatorio
    # REAL: numero decimal para el precio
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS productos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            precio REAL NOT NULL,
            descripcion TEXT,
            stock INTEGER DEFAULT 0
        )
    ''')
    
    # Insertamos algunos datos de ejemplo si la tabla esta vacia
    cursor.execute('SELECT COUNT(*) FROM productos')
    if cursor.fetchone()[0] == 0:
        # Datos iniciales
        productos_iniciales = [
            ('Laptop', 1200, 'Laptop de alta gama', 10),
            ('Mouse', 25, 'Mouse inalambrico', 50),
            ('Teclado', 75, 'Teclado mecanico RGB', 30)
        ]
        
        # executemany() ejecuta la misma consulta con diferentes valores
        cursor.executemany(
            'INSERT INTO productos (nombre, precio, descripcion, stock) VALUES (?, ?, ?, ?)',
            productos_iniciales
        )
    
    # commit() guarda los cambios en la base de datos
    db.commit()
    db.close()
    print("Base de datos inicializada correctamente!")

# Ejecutamos la inicializacion
init_db()

---

## 5. Operaciones CRUD con Base de Datos

Ahora implementaremos todas las operaciones CRUD usando la base de datos SQLite.

### 5.1 READ - Obtener Productos de la Base de Datos

In [ ]:
from flask import Flask, jsonify, g
import sqlite3

# GET: Obtener todos los productos desde la base de datos
@app.route('/api/productos', methods=['GET'])
def obtener_productos_db():
    # Obtenemos la conexion a la base de datos
    db = get_db()
    cursor = db.cursor()
    
    # Ejecutamos una consulta SELECT para obtener todos los productos
    cursor.execute('SELECT * FROM productos')
    
    # fetchall() devuelve todas las filas como una lista
    filas = cursor.fetchall()
    
    # Convertimos las filas (Row objects) a diccionarios
    # dict(fila) convierte un Row object a un diccionario
    productos = [dict(fila) for fila in filas]
    
    return jsonify(productos)

# GET: Obtener un producto especifico por ID
@app.route('/api/productos/<int:id>', methods=['GET'])
def obtener_producto_db(id):
    db = get_db()
    cursor = db.cursor()
    
    # Usamos ? como placeholder para evitar SQL injection
    # Los valores se pasan como una tupla en el segundo parametro
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    
    # fetchone() devuelve solo una fila, o None si no hay resultados
    fila = cursor.fetchone()
    
    if fila:
        return jsonify(dict(fila))
    return jsonify({'error': 'Producto no encontrado'}), 404

print("Endpoints READ con base de datos configurados!")

### 5.2 CREATE - Insertar Nuevos Productos en la Base de Datos

In [ ]:
from flask import request

# POST: Crear un nuevo producto en la base de datos
@app.route('/api/productos', methods=['POST'])
def crear_producto_db():
    # Obtenemos los datos JSON del cuerpo de la peticion
    nuevo_producto = request.get_json()
    
    # Validacion de campos requeridos
    if not nuevo_producto or 'nombre' not in nuevo_producto or 'precio' not in nuevo_producto:
        return jsonify({'error': 'Faltan datos requeridos: nombre y precio'}), 400
    
    # Obtenemos la conexion a la base de datos
    db = get_db()
    cursor = db.cursor()
    
    # Insertamos el nuevo producto
    # Usamos get() para campos opcionales con valores por defecto
    cursor.execute(
        'INSERT INTO productos (nombre, precio, descripcion, stock) VALUES (?, ?, ?, ?)',
        (
            nuevo_producto['nombre'],
            nuevo_producto['precio'],
            nuevo_producto.get('descripcion', ''),  # cadena vacia si no se envia
            nuevo_producto.get('stock', 0)  # 0 si no se envia
        )
    )
    
    # Guardamos los cambios en la base de datos
    db.commit()
    
    # lastrowid contiene el ID del registro que acabamos de insertar
    nuevo_id = cursor.lastrowid
    
    # Consultamos el producto recien creado para devolverlo completo
    cursor.execute('SELECT * FROM productos WHERE id = ?', (nuevo_id,))
    producto_creado = dict(cursor.fetchone())
    
    return jsonify(producto_creado), 201

print("Endpoint CREATE con base de datos configurado!")

### 5.3 UPDATE - Actualizar Productos en la Base de Datos

In [ ]:
# PUT: Actualizar un producto existente en la base de datos
@app.route('/api/productos/<int:id>', methods=['PUT'])
def actualizar_producto_db(id):
    # Obtenemos los nuevos datos
    datos_actualizados = request.get_json()
    
    db = get_db()
    cursor = db.cursor()
    
    # Primero verificamos que el producto existe
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto_actual = cursor.fetchone()
    
    if not producto_actual:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    # Convertimos a diccionario para facilitar el acceso
    producto_dict = dict(producto_actual)
    
    # Actualizamos solo los campos que se enviaron
    # Si no se envio un campo, mantenemos el valor actual
    nuevo_nombre = datos_actualizados.get('nombre', producto_dict['nombre'])
    nuevo_precio = datos_actualizados.get('precio', producto_dict['precio'])
    nueva_descripcion = datos_actualizados.get('descripcion', producto_dict['descripcion'])
    nuevo_stock = datos_actualizados.get('stock', producto_dict['stock'])
    
    # Ejecutamos el UPDATE
    cursor.execute(
        'UPDATE productos SET nombre = ?, precio = ?, descripcion = ?, stock = ? WHERE id = ?',
        (nuevo_nombre, nuevo_precio, nueva_descripcion, nuevo_stock, id)
    )
    
    # Guardamos los cambios
    db.commit()
    
    # Consultamos el producto actualizado para devolverlo
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto_actualizado = dict(cursor.fetchone())
    
    return jsonify(producto_actualizado), 200

print("Endpoint UPDATE con base de datos configurado!")

### 5.4 DELETE - Eliminar Productos de la Base de Datos

In [ ]:
# DELETE: Eliminar un producto de la base de datos
@app.route('/api/productos/<int:id>', methods=['DELETE'])
def eliminar_producto_db(id):
    db = get_db()
    cursor = db.cursor()
    
    # Verificamos que el producto existe antes de eliminarlo
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto = cursor.fetchone()
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    # Eliminamos el producto
    cursor.execute('DELETE FROM productos WHERE id = ?', (id,))
    
    # Guardamos los cambios
    db.commit()
    
    # Devolvemos 204 No Content (eliminacion exitosa sin cuerpo de respuesta)
    return '', 204

print("Endpoint DELETE con base de datos configurado!")

**COPIA EL SIGUIENTE CODIGO COMPLETO A `app.py` PARA TENER LA API COMPLETA CON BASE DE DATOS:**

In [ ]:
# CODIGO PARA app.py - VERSION 3: CRUD Completo con SQLite
from flask import Flask, jsonify, request, g
import sqlite3

app = Flask(__name__)

# Configuracion de la base de datos
DATABASE = 'productos.db'

# Funcion para obtener conexion a la base de datos
def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect(DATABASE)
        g.db.row_factory = sqlite3.Row
    return g.db

# Cerrar conexion al finalizar cada peticion
@app.teardown_appcontext
def close_db(error):
    db = g.pop('db', None)
    if db is not None:
        db.close()

# Inicializar la base de datos
def init_db():
    db = sqlite3.connect(DATABASE)
    cursor = db.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS productos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            precio REAL NOT NULL,
            descripcion TEXT,
            stock INTEGER DEFAULT 0
        )
    ''')
    
    cursor.execute('SELECT COUNT(*) FROM productos')
    if cursor.fetchone()[0] == 0:
        productos_iniciales = [
            ('Laptop', 1200, 'Laptop de alta gama', 10),
            ('Mouse', 25, 'Mouse inalambrico', 50),
            ('Teclado', 75, 'Teclado mecanico RGB', 30)
        ]
        cursor.executemany(
            'INSERT INTO productos (nombre, precio, descripcion, stock) VALUES (?, ?, ?, ?)',
            productos_iniciales
        )
    
    db.commit()
    db.close()

# Ruta principal
@app.route('/')
def inicio():
    return jsonify({
        'mensaje': 'API de Productos con SQLite',
        'endpoints': {
            'GET /api/productos': 'Obtener todos los productos',
            'GET /api/productos/<id>': 'Obtener un producto especifico',
            'POST /api/productos': 'Crear un nuevo producto',
            'PUT /api/productos/<id>': 'Actualizar un producto',
            'DELETE /api/productos/<id>': 'Eliminar un producto'
        }
    })

# GET: Obtener todos los productos
@app.route('/api/productos', methods=['GET'])
def obtener_productos():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('SELECT * FROM productos')
    filas = cursor.fetchall()
    productos = [dict(fila) for fila in filas]
    return jsonify(productos)

# GET: Obtener un producto por ID
@app.route('/api/productos/<int:id>', methods=['GET'])
def obtener_producto(id):
    db = get_db()
    cursor = db.cursor()
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    fila = cursor.fetchone()
    
    if fila:
        return jsonify(dict(fila))
    return jsonify({'error': 'Producto no encontrado'}), 404

# POST: Crear un nuevo producto
@app.route('/api/productos', methods=['POST'])
def crear_producto():
    nuevo_producto = request.get_json()
    
    if not nuevo_producto or 'nombre' not in nuevo_producto or 'precio' not in nuevo_producto:
        return jsonify({'error': 'Faltan datos requeridos: nombre y precio'}), 400
    
    db = get_db()
    cursor = db.cursor()
    
    cursor.execute(
        'INSERT INTO productos (nombre, precio, descripcion, stock) VALUES (?, ?, ?, ?)',
        (
            nuevo_producto['nombre'],
            nuevo_producto['precio'],
            nuevo_producto.get('descripcion', ''),
            nuevo_producto.get('stock', 0)
        )
    )
    
    db.commit()
    nuevo_id = cursor.lastrowid
    
    cursor.execute('SELECT * FROM productos WHERE id = ?', (nuevo_id,))
    producto_creado = dict(cursor.fetchone())
    
    return jsonify(producto_creado), 201

# PUT: Actualizar un producto
@app.route('/api/productos/<int:id>', methods=['PUT'])
def actualizar_producto(id):
    datos_actualizados = request.get_json()
    
    db = get_db()
    cursor = db.cursor()
    
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto_actual = cursor.fetchone()
    
    if not producto_actual:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    producto_dict = dict(producto_actual)
    
    nuevo_nombre = datos_actualizados.get('nombre', producto_dict['nombre'])
    nuevo_precio = datos_actualizados.get('precio', producto_dict['precio'])
    nueva_descripcion = datos_actualizados.get('descripcion', producto_dict['descripcion'])
    nuevo_stock = datos_actualizados.get('stock', producto_dict['stock'])
    
    cursor.execute(
        'UPDATE productos SET nombre = ?, precio = ?, descripcion = ?, stock = ? WHERE id = ?',
        (nuevo_nombre, nuevo_precio, nueva_descripcion, nuevo_stock, id)
    )
    
    db.commit()
    
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto_actualizado = dict(cursor.fetchone())
    
    return jsonify(producto_actualizado), 200

# DELETE: Eliminar un producto
@app.route('/api/productos/<int:id>', methods=['DELETE'])
def eliminar_producto(id):
    db = get_db()
    cursor = db.cursor()
    
    cursor.execute('SELECT * FROM productos WHERE id = ?', (id,))
    producto = cursor.fetchone()
    
    if not producto:
        return jsonify({'error': 'Producto no encontrado'}), 404
    
    cursor.execute('DELETE FROM productos WHERE id = ?', (id,))
    db.commit()
    
    return '', 204

if __name__ == '__main__':
    # Inicializamos la base de datos antes de ejecutar la app
    init_db()
    print('Base de datos inicializada')
    print('Servidor corriendo en http://localhost:5000')
    app.run(debug=True, host='0.0.0.0', port=5000)

---

## 6. Probando la API con Python (requests)

Vamos a crear funciones para probar todos los endpoints desde Python usando la libreria requests.

In [ ]:
# Instalamos requests si no esta disponible
!pip install requests

In [ ]:
import requests
import json

# URL base de nuestra API
BASE_URL = 'http://localhost:5000/api/productos'

# Funcion para probar GET (obtener todos los productos)
def test_obtener_productos():
    print("\n=== TEST: Obtener todos los productos ===")
    response = requests.get(BASE_URL)
    print(f"Status Code: {response.status_code}")
    print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
    return response.json()

# Funcion para probar GET (obtener un producto especifico)
def test_obtener_producto(id):
    print(f"\n=== TEST: Obtener producto con ID {id} ===")
    response = requests.get(f"{BASE_URL}/{id}")
    print(f"Status Code: {response.status_code}")
    print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
    return response.json()

# Funcion para probar POST (crear un producto)
def test_crear_producto():
    print("\n=== TEST: Crear un nuevo producto ===")
    nuevo_producto = {
        'nombre': 'Monitor 4K',
        'precio': 450,
        'descripcion': 'Monitor 27 pulgadas 4K UHD',
        'stock': 15
    }
    response = requests.post(BASE_URL, json=nuevo_producto)
    print(f"Status Code: {response.status_code}")
    print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
    return response.json()

# Funcion para probar PUT (actualizar un producto)
def test_actualizar_producto(id):
    print(f"\n=== TEST: Actualizar producto con ID {id} ===")
    datos_actualizacion = {
        'precio': 1150,
        'stock': 8
    }
    response = requests.put(f"{BASE_URL}/{id}", json=datos_actualizacion)
    print(f"Status Code: {response.status_code}")
    print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
    return response.json()

# Funcion para probar DELETE (eliminar un producto)
def test_eliminar_producto(id):
    print(f"\n=== TEST: Eliminar producto con ID {id} ===")
    response = requests.delete(f"{BASE_URL}/{id}")
    print(f"Status Code: {response.status_code}")
    if response.status_code == 204:
        print("Producto eliminado exitosamente (sin contenido en respuesta)")
    return response.status_code

print("Funciones de testing creadas!")
print("IMPORTANTE: Asegurate de que app.py este corriendo antes de ejecutar los tests")

In [ ]:
# Ejecutar todos los tests
# NOTA: app.py debe estar corriendo en otro terminal

try:
    # Test 1: Obtener todos los productos
    productos = test_obtener_productos()
    
    # Test 2: Obtener un producto especifico
    test_obtener_producto(1)
    
    # Test 3: Crear un nuevo producto
    nuevo = test_crear_producto()
    nuevo_id = nuevo.get('id')
    
    # Test 4: Actualizar el producto creado
    if nuevo_id:
        test_actualizar_producto(nuevo_id)
    
    # Test 5: Obtener todos los productos de nuevo (para ver cambios)
    test_obtener_productos()
    
    # Test 6: Eliminar el producto creado
    if nuevo_id:
        test_eliminar_producto(nuevo_id)
    
    # Test 7: Verificar que se elimino
    test_obtener_productos()
    
    print("\n=== TODOS LOS TESTS COMPLETADOS ===")
    
except requests.exceptions.ConnectionError:
    print("ERROR: No se puede conectar a la API")
    print("Asegurate de que app.py este corriendo en otro terminal con: python app.py")

---

## 7. Resumen y Proximos Pasos

### Lo que hemos aprendido:
1. Crear una aplicacion Flask basica
2. Definir endpoints con diferentes metodos HTTP (GET, POST, PUT, DELETE)
3. Manejar JSON en peticiones y respuestas
4. Integrar SQLite para persistencia de datos
5. Implementar operaciones CRUD completas
6. Probar APIs con diferentes herramientas

### Proximos pasos para mejorar tu API:
1. **Validacion de datos**: Usar bibliotecas como marshmallow o pydantic
2. **Autenticacion**: Implementar JWT o OAuth2
3. **Paginacion**: Para endpoints que devuelven muchos datos
4. **Manejo de errores**: Crear manejadores personalizados de errores
5. **CORS**: Permitir peticiones desde el frontend
6. **Documentacion**: Usar Flask-RESTX o Swagger
7. **Testing**: Escribir tests automatizados con pytest
8. **Deployment**: Desplegar en produccion con Gunicorn y Nginx

### Recursos adicionales:
- Tutorial completo de Flask: https://flask.palletsprojects.com/en/3.0.x/tutorial/
- Flask Mega-Tutorial: https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world
- REST API Best Practices: https://restfulapi.net/
- SQLAlchemy (ORM para Flask): https://flask-sqlalchemy.palletsprojects.com/

### Ejercicios propuestos:
1. Anadir un endpoint para buscar productos por nombre
2. Implementar filtrado por rango de precios
3. Crear una nueva tabla de categorias y relacionarla con productos
4. Anadir validacion para que el precio no sea negativo
5. Implementar un sistema de logs para registrar todas las operaciones